In [ ]:
!pip install gtts playsound SpeechRecognition replicate pygame

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=bf6aadc2a9b9b2fbc47ec6e1dc3b2f3d76b71a7d3da8727ccf6b14ab4694066b
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gtts playsound SpeechRecognition replicate pygame
import os
import replicate
from gtts import gTTS
import speech_recognition as sr
import pygame
import time

# Set up Replicate API token directly
api_token = 'Your_Replicate_api_key'  # Replace with your actual API token

# Initialize Replicate client
replicate_client = replicate.Client(api_token=api_token)

def speak_text(text):
    """Convert text to speech and play it."""
    tts = gTTS(text=text, lang='en')
    audio_file = "temp_audio.mp3"

    try:
        tts.save(audio_file)

        # Check if the file exists and is not empty
        if os.path.exists(audio_file) and os.path.getsize(audio_file) > 0:
            print(f"Playing audio file: {audio_file}")

            # Initialize pygame mixer
            pygame.mixer.init()
            pygame.mixer.music.load(audio_file)
            pygame.mixer.music.play()

            while pygame.mixer.music.get_busy():
                time.sleep(1)
        else:
            print(f"Audio file is missing or empty: {audio_file}")

    except Exception as e:
        print(f"Error during TTS conversion or playback: {e}")




def recognize_speech_from_file(file_path):
    """Recognize speech from an audio file."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        print("Processing audio file...")
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            print(f"Transcription: {text}")
            return text
        except sr.UnknownValueError:
            print("Sorry, the audio was not understood.")
            return None
        except sr.RequestError as e:
            print(f"Could not request results; {e}")
            return None

def get_response(prompt):
    """Get a response from the Llama model using Replicate's API."""
    model_ref = "meta/meta-llama-3-70b-instruct"  # Ensure this is the correct model reference

    llama_response = ''

    try:
        # Stream the response from Replicate
        for event in replicate_client.run(
            model_ref,
            input={
                "top_k": 0,
                "top_p": 0.9,
                "prompt": prompt,
                "max_tokens": 512,
                "min_tokens": 0,
                "temperature": 0.6,
                "length_penalty": 1,
                "prompt_template": "{prompt}",
                "presence_penalty": 1.15,
                "log_performance_metrics": False
            },
            stream=True
        ):
            # Collect text chunks
            if isinstance(event, str):
                llama_response += event
            else:
                print(f"Unexpected event format: {event}")

        return llama_response.strip()

    except Exception as error:
        print(f"Error parsing response: {error}")
        return "An error occurred while getting a response."

def main():
    # Specify the path to your audio file
    audio_file_path = "/content/drive/MyDrive/question.wav"  # Replace with your audio file path

    # Recognize speech from the audio file
    user_input = recognize_speech_from_file(audio_file_path)
    if user_input is None:
        return

    # Get response from the Llama model
    response = get_response(user_input)

    # Output the response
    print(f"Assistant: {response}")
    speak_text(response)

if __name__ == "__main__":
    main()

pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Processing audio file...
Transcription: what news sports or events will be introduced in the 2024 Paris Olympics
Assistant: ?
The International Olympic Committee (IOC) has announced that breakdancing, surfing, skateboarding, and sport climbing will be added to the Olympic program for the 2024 Paris Olympics.

Breakdancing, also known as breaking, will make its Olympic debut with 16 athletes competing in men's and women's events. Surfing will feature 20 athletes competing in men's and women's shortboard events. Skateboarding will include 20 athletes competing in men's and women's park events. Sport climbing will feature 20 athletes competing in men's and women's combined events.

These new sports were chosen to appeal to a younger generation and to increase the Olympic Games' relevance and attractiveness to a broader audience.
Playing audio file: temp_audio.mp3
Error during 